In [1]:
VERSION = '99_v1_02_24'

In [2]:
model_list = [    
    {'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':1}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    {'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':1}, #deberta-base, seed100, split00_v2_07
    
    {'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':1}, #longformer-large, seed100, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v2_07/oof_df.csv', 'w':1}, #deberta-large, seed200, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v1_04/oof_df.csv', 'w':1}, #deberta-v3-large, seed200, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v1_05/oof_df.csv', 'w':1}, #deberta-xlarge, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_22/oof_df.csv', 'w':1}, #deberta-v2-xlarge, seed100, split00_v2_07
    
    #{'path':'../../19_RNN/exp/result/19_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    
    {'path':'../../20_WoSpanDet/exp/result/20_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, huggingface pre
    #{'path':'../../20_WoSpanDet/exp/result/20_v1_02/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07, huggingface pre
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
    pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
    pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5838


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,...,Effective_5,Ineffective_6,Adequate_6,Effective_6,Ineffective_7,Adequate_7,Effective_7,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.004022,0.112371,0.008607,1,0.106507,...,0.005047,0.001372,0.120747,0.002881,0.009771,0.112351,0.002878,0.037230,0.903354,0.059416
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.012844,0.110929,0.001228,1,0.119425,...,0.003243,0.014672,0.105444,0.004884,0.006497,0.116235,0.002269,0.077773,0.896576,0.025651
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.034133,0.090335,0.000532,1,0.324790,...,0.002259,0.006496,0.114376,0.004128,0.022119,0.101927,0.000955,0.162262,0.821709,0.016029
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.026096,0.097522,0.001382,1,0.248233,...,0.001734,0.016041,0.107312,0.001647,0.024509,0.099422,0.001070,0.217692,0.769340,0.012968
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.039354,0.085156,0.000490,1,0.383832,...,0.001299,0.028141,0.095388,0.001471,0.040726,0.083482,0.000792,0.297290,0.693287,0.009424
